#### Connect to Workspace

In [1]:
# Import Libraries 
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential 
from azure.ai.ml import MLClient 
from azure.ai.ml.entities import AzureBlobDatastore, AccountKeyConfiguration, AmlCompute

In [2]:
# get connection to workspace 
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()
    
# get token 
ws = MLClient.from_config(credential=credential)
print(ws)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fccd770abf0>,
         subscription_id=567f42ad-44d7-4850-aada-1d69d5b9aae9,
         resource_group_name=dp-100rg,
         workspace_name=projectomegadev)


##### Connect Data Lake to Work Station

In [5]:
# connect a Gen2 Storage account to workspace 

ds = AzureBlobDatastore(
    name="ws_datastore",
    account_name="projectomegadatalake2",
    container_name="data",
    description="Data Store for ML",
    credentials=AccountKeyConfiguration(
        account_key=""
    )    
)
# create datastore
ws.create_or_update(ds)

In [3]:
# list datastores connected to workspace 
ds = ws.datastores.list()
for each in ds:
    print(each.name)

ws_datastore
omegadatalake2
workspaceartifactstore
workspaceworkingdirectory
workspaceblobstore
workspacefilestore


#### Create Compute Cluster

In [8]:
# create name and assign to the compute cluster
cpu_compute_target = "compute-cluster"

try:
    # check if the compute target already exists
    cpu_cluster = ws.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS11_V2",
        min_instances=0,
        max_instances=2,
        idle_time_before_scale_down=120,
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ws.compute.begin_create_or_update(cpu_cluster)

Creating a new cpu compute target...


In [4]:
# confirm compute cluster 
name = ws.compute.get("compute-cluster")
print(f"You already have a cluster named {name}.")

You already have a cluster named enable_node_public_ip: true
id: /subscriptions/567f42ad-44d7-4850-aada-1d69d5b9aae9/resourceGroups/dp-100rg/providers/Microsoft.MachineLearningServices/workspaces/projectomegadev/computes/compute-cluster
idle_time_before_scale_down: 120
location: eastus
max_instances: 2
min_instances: 0
name: compute-cluster
network_settings: {}
provisioning_state: Succeeded
size: STANDARD_DS11_V2
ssh_public_access_enabled: true
tier: dedicated
type: amlcompute
.
